# Transformer Drills

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.datasets import mnist
import logging
import matplotlib.pyplot as plt
import math 
import random

## Configure Log Settings


In [2]:
logging.basicConfig(filename = "logs.log", format = "%(asctime)s -- %(message)s", datefmt='%m/%d/%Y %I:%M:%S %p', level = logging.INFO)

## Load Dataset

In [3]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar100.load_data()
logging.info("Dataset Upload successfully")
logging.info(f"X training set shape:{X_train.shape}")
logging.info(f"y training set shape:{y_train.shape}")
logging.info(f"X test set shape:{X_test.shape}")
logging.info(f"y training set shape:{y_test.shape}")

## HyperParameters

In [4]:
learning_rate = 1e-3
batch_size = 256
image_size = 72 
patch_size = 6
num_patches = int(image_size/patch_size)**2
patch_plot_size = int(image_size/patch_size)
epochs = 1000
projection_dim = 512




## Functions


In [5]:
#Normalize pixels
def normalize_images(images):
    images = np.asarray(images)/255.0
    return images

X_train = normalize_images(X_train)
X_test = normalize_images(X_test)

In [6]:
def resize_images(images):
    resized_batch = [tf.image.resize(tf.convert_to_tensor(image), size = (image_size, image_size)) for image in images]
    resized_batch = tf.convert_to_tensor(resized_batch)

    return resized_batch
X_train = resize_images(X_train)


2023-02-28 15:42:42.505908: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
X_train.shape

TensorShape([50000, 72, 72, 3])

In [8]:
def create_patches(images, plot_sample = True):
    if plot_sample:
        #Pick a random image and resize to desired shape
        sample_image = images[random.randint(0, len(images))]
        sample_image = tf.image.resize(tf.convert_to_tensor(sample_image), size = (image_size, image_size))
        logging.info(f"Sample Image Shape: {sample_image.shape}")
        
        # Plotting Original Image
        fig = plt.figure(figsize=(7,7))
        fig.add_subplot()
        plt.imshow(sample_image)
        
        #Plotting Patched Image
        fig = plt.figure(figsize=(7,7))
        #Output of extract_patches method is a four dimensional tensor with shape (1, number of columns, number of rows, number of elements per patch)
        patched_image = tf.image.extract_patches(tf.expand_dims(sample_image,0),
                                                sizes = [1, patch_size, patch_size, 1],
                                                strides = [1, patch_size,patch_size, 1],
                                                rates = [1,1,1,1],
                                                padding = "VALID") 
        patched_image = tf.reshape(patched_image, (1,-1,patched_image.shape[-1])) # Output is reshape to be (1 image, number of patches, number of elements per patch)
        for i, ax in enumerate(range(patched_image.shape[1])): #For loop to display each patch in the form of the original image
            fig.add_subplot(patch_plot_size, patch_plot_size, i + 1)
            plt.imshow(tf.reshape(patched_image[0][i], (patch_size,patch_size,3)))
            plt.axis("off")
    patched_batch = tf.image.extract_patches(images,
                                                sizes = [1, patch_size, patch_size, 1],
                                                strides = [1, patch_size,patch_size, 1],
                                                rates = [1,1,1,1],
                                                padding = "VALID")
    patched_batch = tf.reshape(patched_batch, (patched_batch.shape[0], -1, patched_batch.shape[-1]))
    return patched_batch

In [15]:
def positional_encoding(patched_batch):
    patch_projection = keras.layers.Dense(projection_dim)
    position_embedding = keras.layers.Embedding(input_dim = num_patches, output_dim = projection_dim)
    position = tf.range(start=0, limit=num_patches)
    
    encoded = [patch_projection(patched_image) + position_embedding(position) for patched_image in patched_batch]

    return tf.convert_to_tensor(encoded)

In [16]:
patched_batch = create_patches(X_train, False)
positional_encoding(patched_batch)

<tf.Tensor: shape=(50000, 144, 512), dtype=float32, numpy=
array([[[-0.49073076, -0.51930964, -0.17580834, ...,  0.16710979,
          1.5399585 , -0.9312647 ],
        [-0.41420972, -0.5740608 , -0.19866464, ...,  0.16266589,
          1.4698322 , -0.9766863 ],
        [-0.43352097, -0.54207873, -0.19147143, ...,  0.16187772,
          1.5473839 , -0.9648883 ],
        ...,
        [-0.1787727 , -0.19783251, -0.08460569, ...,  0.05926875,
          0.63588905, -0.3523957 ],
        [-0.17349389, -0.0281522 , -0.11252379, ...,  0.07671636,
          0.28457972, -0.07099849],
        [-0.05283094, -0.23042354,  0.06832343, ...,  0.14822179,
          0.41385093, -0.33758938]],

       [[-0.49215955, -0.51606876, -0.17321283, ...,  0.16896762,
          1.5359584 , -0.93404454],
        [-0.41775727, -0.5729431 , -0.19383645, ...,  0.16258764,
          1.4634867 , -0.98007804],
        [-0.43706852, -0.540961  , -0.18664323, ...,  0.16179948,
          1.5410384 , -0.96828   ],
        